# Sensitivity Analysis
There are two types of sensitivity analysis:
1. **Local sensitivity analysis (LSA)**: Performed by varying uncertain model parameters around specific (initial) values. 
    * The goal is to explore how small changes in input variables influence changes in model output.    
    * Assumes that input variables are independent (they do not interact).
    * If this assumption is false, LSA may underestimate the influence that the inputs have on output metrics.    
    * E.g. One-at-a-time analysis 
2. **Global sensitivity analysis (GSA)**: Performed by varying uncertain model input within the entire feasible space of potential input values. 
   * Reveals the global and interactive effects of each input variable on model output.
   * *Variance-based GSA*: Analysis of model output variance can determine how sensitive it is to its input variables (e.g. Sobol SA, Analysis of Variance (ANOVA))
   * *[Moment-independent GSA](https://waterprogramming.wordpress.com/2020/05/18/information-theory-and-moment-independent-sensitivity-indices/)*: Compares entire unconditional and conditional probability distributions of input variables and model output values to determine model sensitivity to an input variable (e.g. Delta Moment-Independent SA, Pianosi and Wagener method (PAWN))

More information on sensitivity analysis and why we perform it can be found [here](https://uc-ebook.org/docs/html/3_sensitivity_analysis_the_basics.html).

## Import the key libraries
Let's begin by importing some key libraries. Some might look familiar, with the exception of SALib. SALib is a Python library that contains commonly-used sensitivity analysis method and useful uncertainty sampling functions. You can install the library [here](https://github.com/SALib/SALib) by cloning the Git repository, or by entering `pip install SALib` into your command line.

In [4]:
# import function to generate uncertainty samples within a specified range using the Saltelli sampling scheme
from SALib.sample import saltelli   
# import function to perform different types of sensitivity analysis
from SALib.analyze import sobol
from SALib.analyze import delta
# import all other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## One-at-a-Time Sensitivity Analysis
Problem description here

In [ ]:
### code for OAT

## Sobol Sensivity Analysis here
Include default exercise as well as TBW-related exercise

In [ ]:
### code for Sobol

## Delta Moment-Independent Sensitivity Analysis
Problem description here

In [5]:
def find_bounds(input_file):
    """
    Finds the founds of the decision variables or DU factor multipliers.
    Parameters
    ----------
    input_file : numpy matrix
        A numpy matrix that specifies the lower and upper bounds of each decision
        variable or DU factor multiplier.
    Returns
    -------
    bounds : tuple
        The lower and upper bound of a decision variable or DU factor multiplier.
    """
    bounds = np.zeros((input_file.shape[1],2), dtype=float)
    for i in range(input_file.shape[1]):
        bounds[i,0] = min(input_file[:,i])
        bounds[i,1] = max(input_file[:,i])

    return 

In [6]:
def delta_sensitivity(dv_du, measured_outcomes, names, mo_names, bounds, compSol_full, rob_objs):
    """
    Main function that performs Delta moment-independent sensitivity analysis
    Writes a csv file to a subfolder named 'delta_[rob_objs]_[mode]/S1_[util]_[compSol].csv'
    
    Parameters
    ----------
    dv_du : numpy matrix
        Contains the float values of the decision variables of each perturbed instance
        of a given compromise solution OR the DU factor multipliers.
    measured_outcomes : numpy matrix
        Contains the float values of either the performance objectives or robustness 
        values, depending on the mode.
    names : list of strings
        Names of all relevant decision variables or DU factor multipliers.
    mo_names : list of strings
        Names of all relevant performance objectives or utilities (for robustness).
    bounds : numpy matrix
        An (len(dv_du) x 2) matrix of the lower and upper bounds of the decision variables
        or DU factor multipliers.
    compSol_full : string
        Longer abbreviation of the compromise solution name
        Social planner: LS98
        Pragmatist: PW113
    rob_objs : string
        Subfolder label indicating if this is sensitivity of robustness or objectives.
    Returns
    -------
    None.
    """
    X = dv_du
    Y = measured_outcomes

    problem = {
        'num_vars': int(dv_du.shape[1]),
        'names': names,
        'bounds': bounds
    }
    print('compSol: ', compSol_full)
    for i in range(measured_outcomes.shape[1]):
        
        
        mo_label = mo_names[i]
        print('obj: ', mo_label)
        
        filename = 'delta_output/delta_' + rob_objs + '_' + mode + '/S1_' + mo_label + '_' + compSol_full + '.csv'
        
        #filename = 'delta_output/delta_base_rdm/S1_' + mo_label + '_' + compSol_full + '.csv'
        S1 = delta.analyze(problem, X, Y[mo_label].values, num_resamples=10, conf_level=0.95, print_to_console=False)
        numpy_S1 = np.array(S1["S1"])
        fileout = pd.DataFrame([names, numpy_S1], index = None, columns = None)
        fileout.to_csv(filename, sep=",")